<a href="https://colab.research.google.com/github/jonesLevin/Machine-Learning/blob/main/FineTuning_SLM_for_Structured_Data_Extractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q trl
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.7 MB/s eta 0:00:00


## Importing Libraries

In [2]:
import torch

from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig


## Model

In [3]:
MODEL_NAME = 'google/gemma-3-270m-it'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# Qunatization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# Loading the model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # quantization_config=bnb_config,
    dtype="auto",
    device_map='auto',
    attn_implementation="eager"
)

# Loading the tokenzer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [5]:
outputs = model(torch.tensor(tokenizer("Hello my name is Levin")['input_ids']).unsqueeze(0).to(DEVICE))
outputs.keys()

odict_keys(['logits', 'past_key_values'])

## Getting the Dataset

In [6]:
dataset = load_dataset('mrdbourke/FoodExtract-1k')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/616k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1420 [00:00<?, ? examples/s]

In [7]:
print(f"[INFO] Number of samples in the dataset: {len(dataset['train'])}")

[INFO] Number of samples in the dataset: 1420


In [8]:
def get_random_idx(dataset):
  """ Returns a random integer index based on the number of samples in the dataset """
  return torch.randint(0, len(dataset), (1,)).item()

random_idx = get_random_idx(dataset['train'])
random_sample = dataset['train'][random_idx]

display(random_sample)

{'sequence': 'A top‑down view of a dark amber bottle of Worcestershire sauce with its label facing up and a slightly ajar cap, a thin glossy stream of the sauce spilling onto a clean white surface, positioned beside a small white ceramic bowl overflowing with glistening pink peppercorns, a few peppercorns scattered around the bowl, and a wooden spoon resting nearby, the soft natural light emphasizing the deep caramel hue of the sauce and the rose‑tinted sparkle of the peppercorns.',
 'image_url': None,
 'class_label': 'food',
 'source': 'gpt-oss-120b',
 'char_len': 471.0,
 'word_count': 78.0,
 'syn_or_real': 'syn',
 'uuid': '09ee82c9-7606-4486-bbab-918e1d892f5a',
 'gpt-oss-120b-label': "{'is_food_or_drink': True, 'tags': ['fp', 'fi'], 'food_items': ['Worcestershire sauce', 'pink peppercorns'], 'drink_items': []}",
 'gpt-oss-120b-label-condensed': 'food_or_drink: 1\ntags: fp, fi\nfoods: Worcestershire sauce, pink peppercorns\ndrinks:',
 'target_food_names_to_use': ['worcestershire_sauce

In [9]:
example_input = random_sample['sequence']
example_output = random_sample['gpt-oss-120b-label']
example_output_condensed = random_sample['gpt-oss-120b-label-condensed']

print(f'[INFO] Input: {example_input}')
print()
print(f'[INFO] Output: {example_output}')
eval(example_output)
print()
print(f'[INFO] Example output condesed:')
display(example_output_condensed)

[INFO] Input: A top‑down view of a dark amber bottle of Worcestershire sauce with its label facing up and a slightly ajar cap, a thin glossy stream of the sauce spilling onto a clean white surface, positioned beside a small white ceramic bowl overflowing with glistening pink peppercorns, a few peppercorns scattered around the bowl, and a wooden spoon resting nearby, the soft natural light emphasizing the deep caramel hue of the sauce and the rose‑tinted sparkle of the peppercorns.

[INFO] Output: {'is_food_or_drink': True, 'tags': ['fp', 'fi'], 'food_items': ['Worcestershire sauce', 'pink peppercorns'], 'drink_items': []}

[INFO] Example output condesed:


'food_or_drink: 1\ntags: fp, fi\nfoods: Worcestershire sauce, pink peppercorns\ndrinks:'

## Formatting the Dataset into LLM Style Inputs and Outputs

In [10]:
def sample_to_conversation(sample):
  return {
      'messages': [
          {'role': 'user', 'content': sample['sequence']},
          {'role': 'system', 'content': sample['gpt-oss-120b-label-condensed']}
      ]
  }

In [11]:
sample_to_conversation(random_sample)

{'messages': [{'role': 'user',
   'content': 'A top‑down view of a dark amber bottle of Worcestershire sauce with its label facing up and a slightly ajar cap, a thin glossy stream of the sauce spilling onto a clean white surface, positioned beside a small white ceramic bowl overflowing with glistening pink peppercorns, a few peppercorns scattered around the bowl, and a wooden spoon resting nearby, the soft natural light emphasizing the deep caramel hue of the sauce and the rose‑tinted sparkle of the peppercorns.'},
  {'role': 'system',
   'content': 'food_or_drink: 1\ntags: fp, fi\nfoods: Worcestershire sauce, pink peppercorns\ndrinks:'}]}

In [12]:
# Map sample to conversation function to the dataset
dataset = dataset.map(sample_to_conversation,
                      batched=False)
dataset['train'][0]

Map:   0%|          | 0/1420 [00:00<?, ? examples/s]

{'sequence': 'A mouth-watering photograph captures a delectable dish centered on a rectangular white porcelain plate, resting on a rustic wooden tabletop indoors. In the background, a wooden cutting board with a long handle subtly enhances the setting. The plate is adorned with several generously-sized, cheese-stuffed peppers that have been roasted to perfection, their blistered skins marked by charred black spots. Split down the middle, the peppers reveal a creamy white cheese filling, enriched with a blend of aromatic herbs. Once stuffed, the peppers have been closed and roasted, achieving a luscious, smoky flavor.\n\nThe dish is elegantly garnished with vibrant cherry tomato halves, freshly chopped green herbs, and delicate sprinkles of small diced red onions. A light, possibly citrus-infused dressing, hinted by a sheen of oil or lime juice, gently coats the ensemble, adding an extra layer of freshness. The meticulous presentation and vivid colors make this image not only a feast fo

In [13]:
# Create a train test split
dataset = dataset['train'].train_test_split(test_size=0.2,
                                            shuffle=False,
                                            seed=42)

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'image_url', 'class_label', 'source', 'char_len', 'word_count', 'syn_or_real', 'uuid', 'gpt-oss-120b-label', 'gpt-oss-120b-label-condensed', 'target_food_names_to_use', 'caption_detail_level', 'num_foods', 'target_image_point_of_view', 'messages'],
        num_rows: 1136
    })
    test: Dataset({
        features: ['sequence', 'image_url', 'class_label', 'source', 'char_len', 'word_count', 'syn_or_real', 'uuid', 'gpt-oss-120b-label', 'gpt-oss-120b-label-condensed', 'target_food_names_to_use', 'caption_detail_level', 'num_foods', 'target_image_point_of_view', 'messages'],
        num_rows: 284
    })
})

## Model with a pipeline

In [15]:
# Load model and use it as a pipeline
pipe = pipeline('text-generation',
                model=model,
                tokenizer=tokenizer)

input_text = [{'role': 'user', 'content': 'Hello my name is Levin. Please reply to me with a machine learning poem'}]

input_prompt = pipe.tokenizer.apply_chat_template(input_text,
                                                  tokenize=False,
                                                  add_generation_prompt=True
)
print(f'[INFO] This is the input prompt: {input_prompt}')

Device set to use cuda:0


[INFO] This is the input prompt: <bos><start_of_turn>user
Hello my name is Levin. Please reply to me with a machine learning poem<end_of_turn>
<start_of_turn>model



In [16]:
default_outputs = pipe(input_prompt, max_new_tokens=256, disable_compile=True)
print(default_outputs[0]['generated_text'][len(input_prompt):])

Okay, Levin, I'm ready. Let's craft a machine learning poem about the feeling of being a small, quiet, and curious observer. Let's start with the opening lines.



 ## Try the Model on One of the Sequences

In [17]:
# Get a random sample
random_idx = get_random_idx(dataset['train'])
random_train_sample = dataset['train'][random_idx]

In [18]:
# Apply chat template to
input_prompt = pipe.tokenizer.apply_chat_template(random_train_sample['messages'][:1],
                                                  tokenize=False,
                                                  add_generation_prompt=True
)
print(f'[INFO] This is the input prompt: {input_prompt}')

[INFO] This is the input prompt: <bos><start_of_turn>user
Online Self Sufficient Vegetable Gardening Course In A Virtual Classroom For One<end_of_turn>
<start_of_turn>model



In [19]:
# Running the default model on the input
default_outputs = pipe(input_prompt, max_new_tokens=256)
print(default_outputs[0]['generated_text'][len(input_prompt):])

The best online self-sufficient vegetable gardening course for one would be a **"Grow Your Own Vegetables with Confidence"** or a **"Sustainable Gardening for Beginners"**.



In [20]:
# View and compare the outputs
print(f'[INFO] Input: \n {input_prompt}')
print(f"[INFO] Output: \n {default_outputs[0]['generated_text'][len(input_prompt):]}")

[INFO] Input: 
 <bos><start_of_turn>user
Online Self Sufficient Vegetable Gardening Course In A Virtual Classroom For One<end_of_turn>
<start_of_turn>model

[INFO] Output: 
 The best online self-sufficient vegetable gardening course for one would be a **"Grow Your Own Vegetables with Confidence"** or a **"Sustainable Gardening for Beginners"**.



## Prompting the Model

In [35]:
prompt_instruction = """Given the following target input text from an image caption, please extract the food and drink items to a list.
If there are no food or drink items, return an empty list.

Return in the following format:
food_items: [item_1, item_2, item_3]
drink_items: [item_4, item_5, item_6]

For example:
Input text: Hello my name is Levin.
Output:
food_items: []
drink_items: []

Example 2:
Input text: A plate if rice cakes, salmon, cottage cheese, and a small cherry tomatoes with a cup of tea.
Output:
food_items: ['rice cakes', 'salmon', 'cottage cheese']
drink_items: ['cup of tea']

Target input text: <targ_input_text>"""


def update_input_message_content(input):
  original_content = input['messages'][:1][0]['content']
  new_content = prompt_instruction.replace('<targ_input_text>', original_content)

  new_input = [{'content': new_content, 'role': 'user'}]

  return new_input

In [36]:
print(f'[INFO] Original content: {random_train_sample['messages'][:1][0]['content']}')
print()
print(f'[INFO] New content with instructions in prompt')
print(update_input_message_content(random_train_sample))

[INFO] Original content: Online Self Sufficient Vegetable Gardening Course In A Virtual Classroom For One

[INFO] New content with instructions in prompt
[{'content': "Given the following target input text from an image caption, please extract the food and drink items to a list. \nIf there are no food or drink items, return an empty list.\n\nReturn in the following format:\nfood_items: [item_1, item_2, item_3]\ndrink_items: [item_4, item_5, item_6]\n\nFor example:\nInput text: Hello my name is Levin.\nOutput: \nfood_items: []\ndrink_items: []\n\nExample 2:\nInput text: A plate if rice cakes, salmon, cottage cheese, and a small cherry tomatoes with a cup of tea.\nOutput: \nfood_items: ['rice cakes', 'salmon', 'cottage cheese']\ndrink_items: ['cup of tea']\n\nTarget input text: Online Self Sufficient Vegetable Gardening Course In A Virtual Classroom For One", 'role': 'user'}]


In [37]:
# Apply chat template
updated_input_prompt = update_input_message_content(random_train_sample)

input_prompt = pipe.tokenizer.apply_chat_template(updated_input_prompt,
                                                  tokenize=False,
                                                  add_generation_prompt=True
)

default_outputs = pipe(input_prompt, max_new_tokens=256)

# View and compare outputs
print(f'[INFO] Input: \n {input_prompt}')
print(f"[INFO] Output: \n {default_outputs[0]['generated_text'][len(input_prompt):]}")

[INFO] Input: 
 <bos><start_of_turn>user
Given the following target input text from an image caption, please extract the food and drink items to a list. 
If there are no food or drink items, return an empty list.

Return in the following format:
food_items: [item_1, item_2, item_3]
drink_items: [item_4, item_5, item_6]

For example:
Input text: Hello my name is Levin.
Output: 
food_items: []
drink_items: []

Example 2:
Input text: A plate if rice cakes, salmon, cottage cheese, and a small cherry tomatoes with a cup of tea.
Output: 
food_items: ['rice cakes', 'salmon', 'cottage cheese']
drink_items: ['cup of tea']

Target input text: Online Self Sufficient Vegetable Gardening Course In A Virtual Classroom For One<end_of_turn>
<start_of_turn>model

[INFO] Output: 
 food_items: []
drink_items: []



## Fine-Tuning the SLM

### Setup SFTConfig

In [ ]:
# Setting up SFTConfig
